In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [3]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [4]:
# ###############################################
#
# --- Helper functions for Linear Regression
#
# ###############################################

def compute_loss(y, tx, w):
    """Calculate the MSE."""
    e = y - tx.dot(w)
    return np.square(e).mean() / 2


def compute_gradient(y, tx, w):
    """Compute the gradient."""
    e = y - tx.dot(w)
    n = len(e)
    return -1/n * tx.T.dot(e)

In [7]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    """Gradient descent algorithm."""
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        grad = compute_gradient(y, tx, w)
        loss = compute_loss(y, tx, w)
        w = w - gamma * grad
        
        # store w and loss
        ws.append(w)
        losses.append(loss)
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))

    return losses, ws


def least_squares_SGD(y, tx, initial_w, batch_size, max_iters, gamma):
    """Stochastic gradient descent algorithm."""
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        for sy, stx in batch_iter(y, tx, batch_size):
            grad = compute_gradient(y, tx, w)
            loss = compute_loss(y, tx, w)
            w = w - gamma * grad
            # store w and loss
            ws.append(w)
            losses.append(loss)
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
            bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))

    return losses, ws

In [8]:
# Linear regression parameters initialization
max_iters = 50
gamma = 1
initial_w = np.zeros((tX.shape[1], 1))

least_squares_GD(y, tX, initial_w, max_iters, gamma)

eiuhiuhiuhriur
eeeeee
ffffff
hey
hey 2
Gradient Descent(0/49): loss=0.5, w0=[ 24.7237 -24.7237 -24.7237 -24.7237 -24.7237 -24.7237  24.7237  24.7237
 -24.7237  24.7237], w1=[ 57.3257 -57.3257 -57.3257 -57.3257 -57.3257 -57.3257  57.3257  57.3257
 -57.3257  57.3257]
eiuhiuhiuhriur
eeeeee
ffffff
hey
hey 2
Gradient Descent(1/49): loss=9700405514956.55, w0=[ 30878910.24489446 -30878910.24489446 -30878910.24489446
 -30878910.24489446 -30878910.24489446 -30878910.24489446
  30878910.24489446  30878910.24489446 -30878910.24489446
  30878910.24489446], w1=[-2.46875728e+08  2.46875728e+08  2.46875728e+08  2.46875728e+08
  2.46875728e+08  2.46875728e+08 -2.46875728e+08 -2.46875728e+08
  2.46875728e+08 -2.46875728e+08]
eiuhiuhiuhriur
eeeeee
ffffff
hey
hey 2
Gradient Descent(2/49): loss=2.7657093157236148e+26, w0=[-1.64196171e+14  1.64196171e+14  1.64196171e+14  1.64196171e+14
  1.64196171e+14  1.64196171e+14 -1.64196171e+14 -1.64196171e+14
  1.64196171e+14 -1.64196171e+14], w1=[ 1.31014295e+15 -1

/Users/sebastiencartier/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in square
  # Remove the CWD from sys.path while we load stuff.


([0.5,
  9700405514956.55,
  2.7657093157236148e+26,
  7.887453527371507e+39,
  2.2494056758152366e+53,
  6.4150311049439934e+66,
  1.8294887634539955e+80,
  5.217479199796535e+93,
  1.4879615411748005e+107,
  4.243485145281709e+120,
  1.210190295913776e+134,
  3.451315374468331e+147,
  9.842731225213987e+160,
  2.807027103013629e+174,
  8.005299521812133e+187,
  2.2830139532719176e+201,
  6.510877821164154e+214,
  1.8568230799191397e+228,
  5.295433342203197e+241,
  1.5101931134407519e+255,
  4.3068868825284514e+268,
  1.228271699414246e+282,
  3.5028813357091797e+295,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  nan,
  nan,
  nan],
 [array([[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
        

In [ ]:
least_squares_SGD(y, tX, initial_w, 1, max_iters, gamma)

In [ ]:
def least_squares(y, tx):
    """calculate the least squares."""
    a = tx.T.dot(tx)
    b = tx.T.dot(y)
    return np.linalg.solve(a, b)


def ridge_regression(y, tx, lambda_):
    """implement ridge regression."""
    m, n = tx.shape[0], tx.shape[1]
    lambda_prime = lambda_ * 2 * m * np.eye(n)
    a = tx.T.dot(tx) + lambda_prime
    b = tx.T.dot(y)
    return np.linalg.solve(a, b)

In [7]:
# ###############################################
#
# --- Helper functions for Logistic Regression
#
# ###############################################

def sigmoid(t):
    """Logistic function"""
    return 1 / (1 + np.exp(-t))


def calculate_loss(y, tx, w):
    """Negative log-likelihood"""
    pred = sigmoid(tx.dot(w))
    a = -y.T.dot(np.log(pred))
    b = (1-y).T.dot(np.log(1-pred))
    return a-b


def calculate_gradient(y, tx, w):
    """compute the gradient of loss."""
    return tx.T.dot(sigmoid(tx.dot(w))-y)


def calculate_hessian(y, tx, w):
    """return the Hessian of the loss function."""
    arr = sigmoid(tx.dot(w)).T[0]
    pred = np.diag(arr)
    S = np.multiply(pred, 1-pred)
    return tx.T.dot(S).dot(tx)

In [8]:
def logistic_regression(y, tx, w):
    """return the loss, gradient, and Hessian."""
    loss = calculate_loss(y, tx, w)
    grad = calculate_gradient(y, tx, w)
    hess = calculate_hessian(y, tx, w)
    return loss, grad, hess


def penalized_logistic_regression(y, tx, w, lambda_):
    """return the loss, gradient."""
    loss = calculate_loss(y, tx, w) + (lambda_ * np.square(np.linalg.norm(w)))
    grad = 2 * lambda_ * w + calculate_gradient(y, tx, w)
    return loss, grad

## Generate predictions and save ouput in csv format for submission:

In [10]:
DATA_TEST_PATH = ''
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = ''
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)